In [4]:
from omegaconf import OmegaConf
from train import do_spectrogram_diffusion
import torchaudio
from diffusion import space_timesteps, SpacedDiffusion, get_named_beta_schedule
from aa_model import AA_diffusion, denormalize_tacotron_mel, normalize_tacotron_mel
import torch

device='cuda:3'
path = '/home/hyc/NS2VC/logs/vc/2024-01-11-14-26-51/model-261.pt'
def load_model(model, ckpt_path):
    data = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(data['model'])
    model = model.to(device)

infer_diffuser = SpacedDiffusion(use_timesteps=space_timesteps(1000, [30]), model_mean_type='epsilon',
                    model_var_type='learned_range', loss_type='mse', betas=get_named_beta_schedule('linear', 1000),
                    conditioning_free=True, conditioning_free_k=2., sampler='dpm++2m')
cfg = OmegaConf.load('config.yaml')
diffusion = AA_diffusion(cfg)
load_model(diffusion, path)

Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is 512 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is 512 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is 512 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is 512 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 512, context_dim is 512 and using 8 heads.
Setting up Memo

In [5]:
import utils
import torchaudio.transforms as T
hmodel = utils.get_hubert_model().to(device)
in_audio_path = 'val_dataset/wangxinling.wav'
refer_audio_path = 'val_dataset/whd.wav'
in_audio, sr_in = torchaudio.load(in_audio_path)
refer_audio, sr_refer = torchaudio.load(refer_audio_path)
in16k = T.Resample(sr_in, 16000)(in_audio).to(device)
in24k = T.Resample(sr_in, 24000)(in_audio).to(device)
cvec = utils.get_hubert_content(hmodel, wav_16k_tensor=in16k[0]).to(device)
cvec = utils.repeat_expand_2d(cvec.squeeze(0), in24k.shape[-1]//256).unsqueeze(0)
refer24k = T.Resample(sr_refer, 24000)(refer_audio).to(device)
spec_process = torchaudio.transforms.MelSpectrogram(
    sample_rate=24000,
    n_fft=1024,
    hop_length=256,
    n_mels=100,
    center=True,
    power=1,
).to(device)
spec = spec_process(refer24k)# 1 100 T
refer = torch.log(torch.clip(spec, min=1e-7)).detach().cpu().to(device)
refer = normalize_tacotron_mel(refer)
mel = do_spectrogram_diffusion(diffusion, infer_diffuser,cvec,refer,temperature=0.8)

load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.hubert_pretraining:current directory is /home/hyc/NS2VC
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_features': 0.1, 'final_dim': 256, 'untie_final_proj'

100%|██████████| 50/50 [00:07<00:00,  6.89it/s]


In [6]:
from vocos import Vocos
mel = mel.detach().cpu()
vocos = Vocos.from_pretrained("charactr/vocos-mel-24khz")
gen = vocos.decode(mel)
from IPython.display import Audio
Audio(gen,rate=24000)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (3): huggingface.co:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (4): huggingface.co:443


: 